<a href="https://colab.research.google.com/github/franklaercio/paul-the-octopus/blob/main/model/paultheoctopus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paul The Octopus Machine Learning

Code by Frank Laércio, Software Developer at CI&T.<br>
frank.junior@ciandt.com [github.com/franklaercio](https://github.com/franklaercio)

![banner.png](https://drive.google.com/uc?export=view&id=1euYvRBbkQtYMslUTwaUUdgcGFc_1gupL)

Content

1.   Importing libraries
2.   Data Analyzing



## Importing libraries

In this section we are import all libraries that need on this project.

*   Numpy for linear algebra
*   Pandas for manipuling data
*   Seaborn for data visualization
*   Matplotlib for data visualization



In [ ]:
import numpy as np 
import pandas as pd
import re
import calendar

import seaborn as sns
sns.set_theme(style="darkgrid")

import matplotlib.pyplot as plt

from google.colab import auth
auth.authenticate_user()

project_id = 'phoenix-cit'
!gcloud config set project {project_id}

MessageError: ignored